## Import packages and API Key

In [29]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")) # Get Key from Env (ignored in git) 
client = OpenAI()
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
from tqdm.auto import tqdm
import fitz  # PyMuPDF
from pathlib import Path

## Load documents in good format

In [30]:
import os
print(os.getcwd())

/workspaces/llm-zoomcamp


In [31]:
with open('mai/context/life_with_hope/structured/steps.json', 'r') as f: 
    steps = json.load(f)

## Create Index Settings for Elasticsearch

### Note: Looks like persists so trips an error if recreating in same instance?

In [39]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "step":        {"type": "integer"},
            "title":       {"type": "text"},
            "text":        {"type": "text"},
            "source":      {"type": "keyword"},
            "page_start":  {"type": "integer"},
            "page_end":    {"type": "integer"},
            "tags":        {"type": "keyword"}
        }
    }
}

index_name = "life-with-hope-steps"

# Create the index (will error if it already exists)
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'life-with-hope-steps'})

In [40]:
for step in tqdm(steps): 
    es_client.index(index=index_name, id=step["step"], document=step)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 59.88it/s]


## Create reusable functions.|

In [73]:
def elastic_search(query, step_number=None):
    must_clause = {
        "multi_match": {
            "query": query,
            "fields": ["title^2", "text"],
            "type": "bool_prefix"
        }
    }

    # Start building the full bool query
    bool_query = {
        "must": must_clause
    }

    # Optionally add a filter for step
    if step_number is not None:
        bool_query["filter"] = {
            "term": {
                "step": step_number
            }
        }

    search_query = {
        "size": 3,
        "query": {
            "bool": bool_query
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs


In [74]:
def build_prompt(query, search_results):
    prompt_template = """
    You are an experienced 12-step Marijuana Anonymous sponsor.
    Answer the question below using only the context from the 'Life with Hope' book.
    If the context doesn't include a direct answer, use your best judgment based on the context to offer a helpful response."
    
    Context:
    {context}
    
    Question:
    {question}
    
    Answer:"""

    context = ""
    for doc in search_results:
        context += f"- Step {doc['step']}: {doc['text']}\n"

    prompt = prompt_template.format(question=query, context=context.strip())
    return prompt


In [75]:
def llm(prompt, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful and spiritually grounded Marijuana Anonymous sponsor. Keep your answers grounded in the 12 steps and the provided context. Avoid speculation."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()


## Create RAG

In [76]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [77]:
query = 'what if I dont believe in a higher power?'

In [78]:
print(rag(query))

If you find yourself struggling with the concept of a Higher Power, it's important to remember that Marijuana Anonymous does not demand any specific beliefs, and there's room within the fellowship for all viewpoints, whether you identify as agnostic, atheist, or theist. The key is to maintain an open mind and a hopeful heart.

The concept of a Higher Power in MA is quite flexible and can be tailored to your own understanding or comfort level. It doesn't have to be a deity; it can be any positive and powerful force that can help guide you in your recovery. For some, this might be the strength of the fellowship, the collective wisdom and support of the group, or simply a state of spirituality or peace that transcends individual will.

In essence, the idea is to find something greater than your own ego and addiction that can aid your recovery journey. So as you work through the Twelve Steps, try to focus on the principles of open-mindedness, humility, and willingness to find what resonate

In [79]:
print(build_prompt(query, elastic_search(query)))



    You are an experienced 12-step Marijuana Anonymous sponsor.
    Answer the question below using only the context from the 'Life with Hope' book.
    If the context doesn't include a direct answer, use your best judgment based on the context to offer a helpful response."

    Context:
    - Step 2: Came to believe that a Power greater than ourselves could restore us to sanity. Step Two was our introduction to the principles of open-mindedness and hope. In Step One we confronted our addiction, admitting that we were powerless over marijuana and that our lives had become unmanageable. We were then left with two alternatives: to stay as we were and continue using marijuana until we died, or to seek spiritual help. Once we admitted our powerlessness, we had to find a power greater than ourselves by which we could live. We knew that our human will alone had never been sufficient to manage our addiction. We began to realize that only a Higher Power could help us. When we came to meetings